# Working with SQUAD Dataset

In [1]:
import pandas as pd
from Utils import get_features
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
ls

QAfeatures.py           __pycache__/            lincoln.txt
Testing_the_Code.ipynb  answers.py              messi.txt
Untitled.ipynb          dennyCode_modified.py
Utils.py                dev-v2.0.json


In [3]:
df = pd.read_json('dev-v2.0.json')

In [4]:
len(df['data'][0]['paragraphs'])

39

In [5]:
def parse_title(df,title_index):
    """Function to parse the SQUAD data structure for one article and return the full text, Questions and Answers.

    Args:
        df (dict): SQUAD data set structure
        title_index (int): Index number for the article you would like to parse

    Returns:
        text (str): The full text of the article (paragraphs are separated by \n)
        Questions (list): A list of all of the questions (marked as possible)
        Answers (list):  A list of all the answers corresponding to the Question list 
    """

    text = '\n'.join([df['data'][title_index]['paragraphs'][i]['context'] for i in range(len(df['data'][title_index]['paragraphs']))])

    Questions = []
    Answers = []

    for i in range(len(df['data'][title_index]['paragraphs'])):
        for j in range(len(df['data'][title_index]['paragraphs'][i]['qas'])):
            if df['data'][title_index]['paragraphs'][i]['qas'][j]['is_impossible'] == True:
                continue
            else:
                Questions.append(df['data'][title_index]['paragraphs'][i]['qas'][j]['question'])
                Answers.append([x['text'] for x in df['data'][title_index]['paragraphs'][i]['qas'][j]['answers']])
    
    return text, Questions, Answers

In [6]:
import random 

def random_index(n, end, start = 0):
    return list(range(start, n)) + list(range(n+1, end))

def filter_questions(vecs,Answers):
    """First figure out if the answer was found in the feature vectors.  Next find and return the answer feature vector and a randomly selected incorrect answer feature vector.
    

    Args:
        vecs (list): list of n feature vectors (from n sentences choosen previously)
        Answers (list): list of answers given by SQUAD

    Returns:
        found_answer (bool): Was the answer found or not?
        correct_feature_vector (array): Feaure vector of the correct answer
        incorrect_feature_vector (array): Feature vector of the incorrect answer
    """

    # Did we find the correct answer?
    check = []
    for vec in vecs:
        check.append(any(item in Answers for item in [str(i) for i in vec.keys()]))
    
    if any(check):
        break_out_flag = False
        for vec in vecs:
            for Answer in set(Answers):
                vec_list = [str(i) for i in vec.keys()]
                if Answer in vec_list:
                    correct_index = vec_list.index(Answer)
                    correct_feature_vector = list(vec.values())[correct_index]
                    incorrect_feature_vector = list(vec.values())[random.choice(random_index(correct_index,len(list(vec.values()))))]
                    while all(incorrect_feature_vector==correct_feature_vector):
                        incorrect_feature_vector = list(vec.values())[random.choice(random_index(correct_index,len(list(vec.values()))))]
                    break_out_flag = True
                    found_answer = True
                    break
            if break_out_flag:
                break
    else:
        correct_feature_vector = np.array([])
        incorrect_feature_vector = np.array([])
        found_answer = False


    return found_answer, correct_feature_vector, incorrect_feature_vector

In [7]:
output_vec

NameError: name 'output_vec' is not defined

In [ ]:
from tqdm.notebook import tqdm


feature_vec = []
output_vec = []
print(len(df['data']))
counter = 0
for j in tqdm(range(len(df['data']))):
    text, Questions, Answers = parse_title(df,j)
    for i, Q in enumerate(Questions):
        try:
            vecs = get_features(text=text,question=Q,num_rel_sentences=3)
        except:
            continue
        try:
            found, correct, incorrect = filter_questions(vecs,Answers[i])
        except:
            continue
        
        if found:
            counter+=1
#             feature_vec.append(correct)
#             output_vec.append(1)

#             feature_vec.append(incorrect)
#             output_vec.append(0)

35


  0%|          | 0/35 [00:00<?, ?it/s]

In [8]:
output_vec

[]

In [131]:


class CustomDataset(Dataset):
    def __init__(self, df, rel_sentences=3):
        
#         feature_vec = []
#         output_vec = []
#         print(len(df['data']))
        
#         for j in range(len(df['data'])):
#             text, Questions, Answers = parse_title(df,j)
#             for i, Q in enumerate(Questions):
#                 try:
#                     vecs = get_features(text=text,question=Q,num_rel_sentences=rel_sentences)
#                 except:
#                     continue
#                 found, correct, incorrect = filter_questions(vecs,Answers[i])
#                 if found:
#                     feature_vec.append(correct)
#                     output_vec.append(1)
                    
#                     feature_vec.append(incorrect)
#                     output_vec.append(0)
            print(j)
                
                
        a = np.array(feature_vec)
        b = np.array(output_vec)

        indices = np.arange(a.shape[0])
        np.random.shuffle(indices)

        self.feature_vec = a[indices]
        self.output_vec = b[indices]
        
    def __len__(self):
        return len(self.output_vec)

    def __getitem__(self, idx):
        x = torch.tensor(self.feature_vec[idx])
        y = torch.tensor(self.output_vec[idx])
        return x , y

In [ ]:
train_set = CustomDataset(df,rel_sentences=3)

35
0
